# COMP30027 Machine Learning Assignment 2

## Description of text features

This notebook describes the pre-computed text features provided for assignment 2. **You do not need to recompute the features yourself for this assignment** -- this information is just for your reference. However, feel free to experiment with different text features if you are interested. If you do want to try generating your own text features, some things to keep in mind:
- There are many different decisions you can make throughout the feature design process, from the text preprocessing to the size of the output vectors. There's no guarantee that the defaults we chose will produce the best possible text features for this classification task, so feel free to experiment with different settings.
- These features must be trained on a training corpus. Generally, the training corpus should not include validation samples, but for the purposes of this assignment we have used the entire non-test set (training+validation) as the training corpus, to allow you to experiment with different validation sets. If you recompute the text features as part of your own model, you should exclude validation samples and retrain on training samples only. Note that if you do N-fold cross-validation, this means generating N sets of features for N different training-validation splits.
- This code may take a long time to run and require a good bit of memory, which is why we are not requiring you to recompute these features yourself. doc2vec in particular is very slow unless you can implement some speed-ups in C.

In [1]:
import numpy as np
import pandas as pd

# read review text
# for DEMONSTRATION PURPOSES, the entire training set will be used to train the models and also as a test set
res_train_review_table = pd.read_csv(r"review_text_train.csv", index_col = False, delimiter = ',', header=0)
train_corpus = res_train_review_table['review']
test = res_train_review_table['review']

## Count vectorizer

A count vectorizer converts documents to vectors which represent word counts. Each column in the output represents a different word and the values indicate the number of times that word appeared in the document. The overall size of a count vector matrix can be quite large (the number of columns is the total number of different words used across all documents in a corpus), but most entries in the matrix are zero (each document contains only a few of all the possible words). Therefore, it is most efficient to represent the count vectors as a sparse matrix.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# preprocess text and compute counts
vocab = CountVectorizer(stop_words='english').fit(train_corpus)

# generate counts for a new set of documents
doc_emb = vocab.transform(test)

## doc2vec

doc2vec methods are an extension of word2vec. word2vec maps words to a high-dimensional vector space in such a way that words which appear in similar contexts will be close together in the space. doc2vec does a similar embedding for multi-word passages. The doc2vec (or Paragraph Vector) method was introduced by:

**Le & Mikolov (2014)** Distributed Representations of Sentences and Documents<br>
https://arxiv.org/pdf/1405.4053v2.pdf

The implementation of doc2vec used for this project is from gensim and documented here:<br>
https://radimrehurek.com/gensim/models/doc2vec.html

The size of the output vector is a free parameter. Most implemementations use around 100-300 dimensions, but the best size depends on the problem you're trying to solve with the embeddings and the number of training samples, so you may wish to try different vector sizes. We provided three sets of doc2vec features, and their dimensions are 50, 100 and 200. The vectors themselves represent directions in a high-dimensional concept space; the columns do not represent specific words or phrases. Values in the vector are continuous real numbers and can be negative.

In [3]:
import gensim

# size of the output vector
vec_size = 20

# function to preprocess and tokenize text
def tokenize_corpus(txt, tokens_only=False):
    for i, line in enumerate(txt):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

# tokenize a training corpus
corpus = list(tokenize_corpus(train_corpus))

# train doc2vec on the training corpus
model = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=40)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

# tokenize new documents
doc = list(tokenize_corpus(test, tokens_only=True))

# generate embeddings for the new documents
doc_emb = np.zeros((len(doc),vec_size))
for i in range(len(doc)):
    doc_emb[i,:] = model.infer_vector(doc[i])